=================
---
## Parallel Computing Code

=================
---

## 1) Defining Inputs and Directories.

In [10]:
% INITIATING INPUTS! 
inputs.nmin1 = 3; 
inputs.nmin2 = 1; 
inputs.OnlyTree = 1; 
inputs.Tria = 0; 
inputs.Dist = 1; 
inputs.MinCylRad = 0.0025;
inputs.ParentCor = 1; 
inputs.TaperCor = 1; 
inputs.GrowthVolCor = 0; 
inputs.GrowthVolFac = 1.5; 
inputs.filter.k = 10;
inputs.filter.radius = 0.00;
inputs.filter.nsigma = 1.5;
inputs.filter.PatchDiam1 = 0.05;
inputs.filter.BallRad1 = 0.075;
inputs.filter.ncomp = 2;
inputs.filter.EdgeLength = 0.004;
inputs.filter.plot = 1;
inputs.name = 'tree'; 
inputs.tree = 1;
inputs.model = 1;
inputs.savemat = 1;
inputs.savetxt = 1; 
inputs.plot = 0;
inputs.disp = 0; 



% User Defined 
% -------------

train_set_on = 40; % ~ 70%
test_set_on  = 16; % ~ 30%
train_set_off = 64; % ~ 70%
test_set_off  = 28; % ~ 30%

no_of_iterations = 2;   % Number TRIALS OR ITERATIONS!
no_of_model_runs = 5;   % Each RUN for each tree will be repeated 5 times and the average DBH will be taken, The reason is that TreemQSM gives different results even for the same inputs.

on_DBH_filepath = 'leaf_on/on_DBH.xlsx'; % YOU NEED TO CHANGE THIS! Directory where DBH observed Data (Excel sheet) is located.
off_DBH_filepath = 'leaf_off/off_DBH.xlsx'; % YOU NEED TO CHANGE THIS! Directory where DBH observed Data (Excel sheet) is located.

train_range_on = ['C' num2str(2) ':C' num2str(train_set_on+1)]; % I am specifying the cells where the DBH data is stored ((from D2 to D21))
test_range_on  = ['C' num2str(train_set_on+2) ':C' num2str(train_set_on+test_set_on+1)]; % Extract the cells containing test data.

train_range_off = ['C' num2str(2) ':C' num2str(train_set_off+1)]; % I am specifying the cells where the DBH data is stored ((from D2 to D21))
test_range_off  = ['C' num2str(train_set_off+2) ':C' num2str(train_set_off+test_set_off+1)]; % Extract the cells containing test data.



## 2) Saving the Point Clouds of the Training Set **LEAF ON** to Perform a Parallel Computing (Make_Models_Parallel).


In [11]:
lasDirectory = 'leaf_on/las'; % This means that the .las files are in the main src directory.
lasFiles = dir(fullfile(lasDirectory, '*.las'));

file_nums = cellfun(@(x) str2double(x(1:end-4)), {lasFiles.name});
[~, sorted_indices] = sort(file_nums);
lasFiles = lasFiles(sorted_indices);
variable_names = cell(1, train_set_on); % This is just to assign variable names to each point cloud before saving. 

for tree = 1:train_set_on
    name = lasFiles(tree).name;
    fullPath = fullfile(lasDirectory, name);
    lasReader = lasFileReader(fullPath);
    ptCloud = readPointCloud(lasReader);
    P = ptCloud.Location - mean(ptCloud.Location);
    var_name = ['P_' num2str(tree)];
    eval([var_name ' = P;']);
    variable_names{tree} = var_name;
end

save('trees_train_on', variable_names{:});

disp("-----------------------")
disp("Saving Trees Completed!")
disp("-----------------------")

-----------------------
Saving Trees Completed!
-----------------------


## 3) Perform Training with parallel computing (Make_Models_Parallel)


In [12]:
rmse_old=1e6; % Initiating a large value for RMSE just for the sake of comparison and storing the smallest RMSE. 
rmse_for_iteration= zeros(no_of_iterations, 1); % Creating an empty array to store RMSE for each trial (for monitoring purposes).

DBH_obs_on = xlsread(on_DBH_filepath, train_range_on); % Here We're importing the array of the observed DBH.From Sheet 2. 

final_params=zeros(4,1); % Because we are dealing with 3 parameters that we need to store and monitor and the fourth is the RMSE score 
for iteration = 1:no_of_iterations % START THE TRIALS LOOP.

    inputs.PatchDiam1    = 0.05 + (0.10 - 0.05) * rand;  % The 0.05 is the lower limit and the 0.10 is the upper limit. [0.05 - 0.10]
    inputs.PatchDiam2Min = 0.02 + (0.06 - 0.02) * rand;  % The 0.02 is the lower limit and the 0.06 is the upper limit. [0.02 - 0.06]
    inputs.PatchDiam2Max = 0.03 + (0.15 - 0.03) * rand;  % The 0.03 is the lower limit and the 0.15 is the upper limit. [0.03 - 0.15]
    inputs.BallRad1 = inputs.PatchDiam1 + 0.015;
    inputs.BallRad2 = inputs.PatchDiam2Max + 0.01;


    QSMs = make_models_parallel( 'trees_train_on' , 'QSMs trees' , no_of_model_runs , inputs );


    % Since the output data structure is complicated, I have to use nested loop to get the output DBH, the below nested loop is just averaging the DBH.
    DBH_qsm=zeros(train_set_on,1);
    

    kk=1;
    for i = 1 : no_of_model_runs : no_of_model_runs*train_set_on
        DBH_array=[];

        c=1;
            
            for j = i:i+no_of_model_runs-1
                DBH = double(QSMs(j).treedata.DBHqsm);
                DBH_array(c)= DBH;
            end
            
            avg_DBH=mean(DBH_array);

        DBH_qsm(kk)=avg_DBH;
        kk = kk+1;
    end

    rmse_score = sqrt(mean((DBH_obs_on - DBH_qsm).^2));
    disp(rmse_score)
    rmse_for_iteration(iteration)=rmse_score; % HERE I am storing the RMSE for each trial over the x trees (training set).
    if rmse_score < rmse_old && inputs.PatchDiam1 > 0 && inputs.PatchDiam2Min>0
        
        % Store values in a 4x1 vector
        final_params = [inputs.PatchDiam1; inputs.PatchDiam2Min; inputs.PatchDiam2Max; rmse_score]; % FINAL RESULTS CONVENTION.
        disp(final_params)
        rmse_old = rmse_score;

    end % End for the if statement.
    disp(['--------Iteration (' num2str(iteration) ') Completed']);
end


Modelling tree 1/40 (P_1):
Modelling tree 2/40 (P_10):
Modelling tree 3/40 (P_11):
Modelling tree 4/40 (P_12):
Modelling tree 5/40 (P_13):
Modelling tree 6/40 (P_14):
Modelling tree 7/40 (P_15):
Modelling tree 8/40 (P_16):
Modelling tree 9/40 (P_17):
Modelling tree 10/40 (P_18):
Modelling tree 11/40 (P_19):
Modelling tree 12/40 (P_2):
Modelling tree 13/40 (P_20):
Modelling tree 14/40 (P_21):
Modelling tree 15/40 (P_22):
Modelling tree 16/40 (P_23):
Modelling tree 17/40 (P_24):
Modelling tree 18/40 (P_25):
Modelling tree 19/40 (P_26):
Modelling tree 20/40 (P_27):
Modelling tree 21/40 (P_28):
Modelling tree 22/40 (P_29):
Modelling tree 23/40 (P_3):
Modelling tree 24/40 (P_30):
Modelling tree 25/40 (P_31):
Modelling tree 26/40 (P_32):
Modelling tree 27/40 (P_33):
Modelling tree 28/40 (P_34):
Modelling tree 29/40 (P_35):
Modelling tree 30/40 (P_36):
Modelling tree 31/40 (P_37):
Modelling tree 32/40 (P_38):
Modelling tree 33/40 (P_39):
Modelling tree 34/40 (P_4):
Modelling tree 35/40 (P_40)

## 4) Performing testing on the ~30% of the trees! (After Parallel Computing)

In [13]:
variable_names_t = cell(1, test_set_on); % This is just to assign variable names to each point cloud before saving. 

cc=1;
for tree = train_set_on+1:train_set_on+test_set_on
    name_t = lasFiles(tree).name;
    fullPath = fullfile(lasDirectory, name_t);
    lasReader_t = lasFileReader(fullPath);
    ptCloud_t = readPointCloud(lasReader_t);
    P_t = ptCloud_t.Location - mean(ptCloud_t.Location);
    var_name_t = ['Pt_' num2str(tree)];
    eval([var_name_t ' = P_t;']);
    variable_names_t{cc} = var_name_t;
    cc = cc+1;
end

save('trees_test_on', variable_names_t{:});

inputs.PatchDiam1    = final_params(1); % NOW 
inputs.PatchDiam2Min = final_params(2); % THEY
inputs.PatchDiam2Max = final_params(3); % ARE FIXED! 
inputs.BallRad1 = inputs.PatchDiam1+0.015;
inputs.BallRad2 = inputs.PatchDiam2Max+0.01;

disp(["Test DBH From Cells:" test_range_on])
DBH_obs_test = xlsread(on_DBH_filepath, test_range_on); % Here We're importing the array of the observed DBH But now for testing.

QSMs_test = make_models_parallel('trees_test_on' , 'QSMs trees test' , no_of_model_runs , inputs);
% TESTING SET =======================================
k=1;
DBH_qsm_test=[];
tot_volume_qsm_test=[];
for i = 1 : no_of_model_runs : no_of_model_runs*test_set_on
DBH_array=[];
tot_volume_array=[];
    c=1;
    for j = i:i+no_of_model_runs-1
        DBH = double(QSMs_test(j).treedata.DBHqsm);
        DBH_array(c)= DBH;
        
        tot_vol = double(QSMs_test(j).treedata.TotalVolume);
        tot_volume_array(c)= tot_vol;
    c=c+1;
    end
    avg_DBH=mean(DBH_array);
    avg_tot_volume=mean(tot_volume_array);
    

DBH_qsm_test(k)=avg_DBH;
tot_volume_qsm_test(k) = avg_tot_volume;
k = k+1;
end

final_test_result(:,1)= 100*DBH_qsm_test;
final_test_result(:,2)= DBH_obs_test;
final_test_result(:,3)= tot_volume_qsm_test;



    "Test DBH From Cells:"    "C42:C57"

Modelling tree 1/16 (Pt_41):
Modelling tree 2/16 (Pt_42):
Modelling tree 3/16 (Pt_43):
Modelling tree 4/16 (Pt_44):
Modelling tree 5/16 (Pt_45):
Modelling tree 6/16 (Pt_46):
Modelling tree 7/16 (Pt_47):
Modelling tree 8/16 (Pt_48):
Modelling tree 9/16 (Pt_49):
Modelling tree 10/16 (Pt_50):
Modelling tree 11/16 (Pt_51):
Modelling tree 12/16 (Pt_52):
Modelling tree 13/16 (Pt_53):
Modelling tree 14/16 (Pt_54):
Modelling tree 15/16 (Pt_55):
Modelling tree 16/16 (Pt_56):


In [14]:
% TRAIN DATA SET =======================================
DBH_obs_train_on = xlsread(on_DBH_filepath, train_range_on); % Here We're importing the array of the observed DBH But now for testing.

k=1;
DBH_qsm_train=[];
tot_volume_qsm_train=[];
for i = 1 : no_of_model_runs : no_of_model_runs*train_set_on
DBH_array=[];
tot_volume_array=[];
    c=1;
    for j = i:i+no_of_model_runs-1
        DBH = double(QSMs(j).treedata.DBHqsm);
        DBH_array(c)= DBH;
        
        tot_vol_train = double(QSMs(j).treedata.TotalVolume);
        tot_vol_train_array(c)= tot_vol_train;
    c=c+1;
    end
    avg_DBH=mean(DBH_array);
    avg_tot_vol_train= mean(tot_vol_train_array);

DBH_qsm_train(k)=avg_DBH;
tot_volume_qsm_train(k)=avg_tot_vol_train;
k = k+1;
end


final_train_result(:,1)= 100*DBH_qsm_train;
final_train_result(:,2)= DBH_obs_train_on;
final_train_result(:,3)= tot_volume_qsm_train;

format shortg
disp("-----------------------")
disp("Final Training Results:")
disp("-----------------------")
disp("       DBH_qsm      DBH_obs   Total_Volume")
disp("       -------      -------   ------------")
disp(final_train_result)
disp("-----------------------")
disp("Final Testing Results: ")
disp("-----------------------")
disp("       DBH_qsm      DBH_obs   Total_Volume")
disp("       -------      -------   ------------")
disp(final_test_result)


-----------------------
Final Training Results:
-----------------------
       DBH_qsm      DBH_obs   Total_Volume
       -------      -------   ------------
       9.2354         29.5       130.28
       56.277         16.5       1323.2
       44.889         19.8       984.43
       15.145         13.8        476.1
       26.174           20       987.41
       26.176            7        958.6
       44.354           17       2787.1
       1.1741         33.9        2.985
       11.983          5.7       142.72
        32.05          5.1       768.35
       56.348         10.2       2106.4
       25.131           40       1024.1
       10.695           31       557.76
       30.208         19.2       636.48
       17.335           30       286.24
        53.87          6.2       5368.3
       11.183         34.7       147.88
       12.304            9       154.36
       40.917          6.1       3947.4
       14.153         32.1       191.89
       132.28         10.9       9214.2
  

## **LEAF OFF=======================================**

In [15]:
lasDirectory = 'leaf_off/las'; % This means that the .las files are in the main src directory.
lasFiles = dir(fullfile(lasDirectory, '*.las'));

file_nums = cellfun(@(x) str2double(x(1:end-4)), {lasFiles.name});
[~, sorted_indices] = sort(file_nums);
lasFiles = lasFiles(sorted_indices);
variable_names = cell(1, train_set_on); % This is just to assign variable names to each point cloud before saving. 

for tree = 1:train_set_off
    name = lasFiles(tree).name;
    fullPath = fullfile(lasDirectory, name);
    lasReader = lasFileReader(fullPath);
    ptCloud = readPointCloud(lasReader);
    P = ptCloud.Location - mean(ptCloud.Location);
    var_name = ['P_' num2str(tree)];
    eval([var_name ' = P;']);
    variable_names{tree} = var_name;
end

save('trees_train_off', variable_names{:});

disp("-----------------------")
disp("Saving Trees Completed!")
disp("-----------------------")

-----------------------
Saving Trees Completed!
-----------------------


In [ ]:
rmse_old=1e6; % Initiating a large value for RMSE just for the sake of comparison and storing the smallest RMSE. 
rmse_for_iteration= zeros(no_of_iterations, 1); % Creating an empty array to store RMSE for each trial (for monitoring purposes).

DBH_obs_off = xlsread(off_DBH_filepath, train_range_off); % Here We're importing the array of the observed DBH.From Sheet 2. 

final_params=zeros(4,1); % Because we are dealing with 3 parameters that we need to store and monitor and the fourth is the RMSE score 
for iteration = 1:no_of_iterations % START THE TRIALS LOOP.

    inputs.PatchDiam1    = 0.05 + (0.10 - 0.05) * rand;  % The 0.05 is the lower limit and the 0.10 is the upper limit. [0.05 - 0.10]
    inputs.PatchDiam2Min = 0.02 + (0.06 - 0.02) * rand;  % The 0.02 is the lower limit and the 0.06 is the upper limit. [0.02 - 0.06]
    inputs.PatchDiam2Max = 0.03 + (0.15 - 0.03) * rand;  % The 0.03 is the lower limit and the 0.15 is the upper limit. [0.03 - 0.15]
    inputs.BallRad1 = inputs.PatchDiam1 + 0.015;
    inputs.BallRad2 = inputs.PatchDiam2Max + 0.01;


    QSMs_off = make_models_parallel( 'trees_train_off' , 'QSMs trees off' , no_of_model_runs , inputs );


    % Since the output data structure is complicated, I have to use nested loop to get the output DBH, the below nested loop is just averaging the DBH.
    DBH_qsm_off=zeros(train_set_off,1);
    

    kk=1;
    for i = 1 : no_of_model_runs : no_of_model_runs*train_set_off
        DBH_array=[];

        c=1;
            
            for j = i:i+no_of_model_runs-1
                DBH = double(QSMs_off(j).treedata.DBHqsm);
                DBH_array(c)= DBH;
            end
            
            avg_DBH=mean(DBH_array);

        DBH_qsm_off(kk)=avg_DBH;
        kk = kk+1;
    end

    rmse_score = sqrt(mean((DBH_obs_off - DBH_qsm_off).^2));
    disp(rmse_score)
    rmse_for_iteration(iteration)=rmse_score; % HERE I am storing the RMSE for each trial over the x trees (training set).
    if rmse_score < rmse_old && inputs.PatchDiam1 > 0 && inputs.PatchDiam2Min>0
        
        % Store values in a 4x1 vector
        final_params = [inputs.PatchDiam1; inputs.PatchDiam2Min; inputs.PatchDiam2Max; rmse_score]; % FINAL RESULTS CONVENTION.
        disp(final_params)
        rmse_old = rmse_score;

    end % End for the if statement.
    disp(['--------Iteration (' num2str(iteration) ') Completed']);
end

Modelling tree 1/64 (P_1):
Modelling tree 2/64 (P_10):
Modelling tree 3/64 (P_11):
Modelling tree 4/64 (P_12):
Modelling tree 5/64 (P_13):


In [8]:
variable_names_t = cell(1, test_set_off); % This is just to assign variable names to each point cloud before saving. 

cc=1;
for tree = train_set_off+1:train_set_off+test_set_off
    name_t = lasFiles(tree).name;
    fullPath = fullfile(lasDirectory, name_t);
    lasReader_t = lasFileReader(fullPath);
    ptCloud_t = readPointCloud(lasReader_t);
    P_t = ptCloud_t.Location - mean(ptCloud_t.Location);
    var_name_t = ['Pt_' num2str(tree)];
    eval([var_name_t ' = P_t;']);
    variable_names_t{cc} = var_name_t;
    cc = cc+1;
end

save('trees_test_off', variable_names_t{:});

inputs.PatchDiam1    = final_params(1); % NOW 
inputs.PatchDiam2Min = final_params(2); % THEY
inputs.PatchDiam2Max = final_params(3); % ARE FIXED! 
inputs.BallRad1 = inputs.PatchDiam1+0.015;
inputs.BallRad2 = inputs.PatchDiam2Max+0.01;

disp(["Test DBH From Cells:" test_range_off])
DBH_obs_test_off = xlsread(off_DBH_filepath, test_range_off); % Here We're importing the array of the observed DBH But now for testing.

QSMs_test_off = make_models_parallel('trees_test_off' , 'QSMs trees test off' , no_of_model_runs , inputs);
% TESTING SET =======================================
k=1;
DBH_qsm_test=[];
tot_volume_qsm_test=[];
for i = 1 : no_of_model_runs : no_of_model_runs*test_set_off
DBH_array=[];
tot_volume_array=[];
    c=1;
    for j = i:i+no_of_model_runs-1
        DBH = double(QSMs_test_off(j).treedata.DBHqsm);
        DBH_array(c)= DBH;
        
        tot_vol = double(QSMs_test_off(j).treedata.TotalVolume);
        tot_volume_array(c)= tot_vol;
    c=c+1;
    end
    avg_DBH=mean(DBH_array);
    avg_tot_volume=mean(tot_volume_array);
    

DBH_qsm_test(k)=avg_DBH;
tot_volume_qsm_test(k) = avg_tot_volume;
k = k+1;
end
final_test_result=[];
final_test_result(:,1)= 100*DBH_qsm_test;
final_test_result(:,2)= DBH_obs_test_off;
final_test_result(:,3)= tot_volume_qsm_test;

    "Test DBH From Cells:"    "C66:C93"

Modelling tree 1/28 (Pt_65):
Incomplete run!!
Incomplete run!!
Incomplete run!!
Incomplete run!!
Incomplete run!!
Modelling tree 2/28 (Pt_66):
Incomplete run!!
Incomplete run!!
Incomplete run!!
Incomplete run!!
Incomplete run!!
Modelling tree 3/28 (Pt_67):
Incomplete run!!
Incomplete run!!
Incomplete run!!
Incomplete run!!
Incomplete run!!
Modelling tree 4/28 (Pt_68):
Incomplete run!!
Incomplete run!!
Incomplete run!!
Incomplete run!!
Incomplete run!!
Modelling tree 5/28 (Pt_69):
Incomplete run!!
Incomplete run!!
Incomplete run!!
Incomplete run!!
Incomplete run!!
Modelling tree 6/28 (Pt_70):
Incomplete run!!
Incomplete run!!
Incomplete run!!
Incomplete run!!
Incomplete run!!
Modelling tree 7/28 (Pt_71):
Incomplete run!!
Incomplete run!!
Incomplete run!!
Incomplete run!!
Incomplete run!!
Modelling tree 8/28 (Pt_72):
Incomplete run!!
Incomplete run!!
Incomplete run!!
Incomplete run!!
Incomplete run!!
Modelling tree 9/28 (Pt_73):
Incomplete run!!
I

Index exceeds the number of array elements (0).



In [9]:
% TRAIN DATA SET =======================================
DBH_obs_train_off = xlsread(off_DBH_filepath, train_range_off); % Here We're importing the array of the observed DBH But now for testing.

k=1;
DBH_qsm_train=[];
tot_volume_qsm_train=[];
for i = 1 : no_of_model_runs : no_of_model_runs*train_set_off
DBH_array=[];
tot_volume_array=[];
    c=1;
    for j = i:i+no_of_model_runs-1
        DBH = double(QSMs(j).treedata.DBHqsm);
        DBH_array(c)= DBH;
        
        tot_vol_train = double(QSMs(j).treedata.TotalVolume);
        tot_vol_train_array(c)= tot_vol_train;
    c=c+1;
    end
    avg_DBH=mean(DBH_array);
    avg_tot_vol_train= mean(tot_vol_train_array);

DBH_qsm_train(k)=avg_DBH;
tot_volume_qsm_train(k)=avg_tot_vol_train;
k = k+1;
end

final_train_result=[];
final_train_result(:,1)= 100*DBH_qsm_train;
final_train_result(:,2)= DBH_obs_train_off;
final_train_result(:,3)= tot_volume_qsm_train;

format shortg
disp("-----------------------")
disp("Final Training Results:")
disp("-----------------------")
disp("       DBH_qsm      DBH_obs   Total_Volume")
disp("       -------      -------   ------------")
disp(final_train_result)
disp("-----------------------")
disp("Final Testing Results: ")
disp("-----------------------")
disp("       DBH_qsm      DBH_obs   Total_Volume")
disp("       -------      -------   ------------")
disp(final_test_result)

Index exceeds the number of array elements (315).

